In [1]:
import tensorflow as tf
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
assert sys.version_info.major == 3
np.random.seed(0)

# 2 - TensorFlow
### 2.1 - Overview

Now we're going to introduce the TensorFlow (TF) framework. This is very much using a sledgehammer to crack a nut, but introducing it in this simple example provides an nice opportunity to explain it. This introduction is largely taken from the [official](https://www.tensorflow.org/get_started/get_started) getting started guide.

TF takes its name from the **tensor** which is a data structure of primative values, which are shaped into an array:

```Python
3 # a rank 0 tensor; this is a scalar with shape []
[1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```

Think. What would a tensor of shape [1, 2, 3, 4] look like? 

Hint: the dimensions are the same as in numpy, so you can play around with that to get a feel for higher-dimensional matrices.

In [2]:
a = np.array([[1., 2., 3.], [4., 5., 6.]])
b = np.array([[[1., 2., 3.]], [[7., 8., 9.]]])
print(a.shape)
print(b.shape)

(2, 3)
(2, 1, 3)


TF is a framework which is designed to facilitate the development of deep learning, and can be thought of as creating a a program in two parts:

1. Building the computational graph
2. Running the computational graph

A computational graph can be though of as a series of **nodes** which take in 0 or more tensors and output a tensor. This could be a node which creates a constant tensor (takes in 0 tensors and outputs 1 tensor), or a node which multiplies tensors together (takes in $n$ tensors, and outputs 1 tensor). In the latter example, these are known as **operations**.

Think of the computational graph as being like a series of pipes, that split and rejoin at various points. Running the computational graph would be like pouring water into the pipes, and seeing it flow around the pipework. Although instead of water, we have data.

Let's start with some simple examples.

### 2.2 - Getting started

Let's build a node which creates a constant:

In [3]:
node_1 = tf.constant(3.0, dtype=tf.float32, name='node_1')

That seems like a lot of code to produce a constant floating point number, and it is, especially if you are used to Python. However it helps to construct the graph, since it is very explicit. From the docs:
```
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)
```

So we can see that there is a slightly more compact way to create the node, if we wanted:

```
node_1 = tf.constant(3.0)
```

However, at this stage it helps to get into the habit of being explicit, _especially_ with naming.

What happens when we print this node? Should we see 3.0?

In [6]:
print(node_1)

Tensor("node_1:0", shape=(), dtype=float32)


Hmm, that's odd. We've got a tensor, with the name of the node, but not much else. This is because we have only completed the first part of the program: building the computational graph. We still need to run the graph.

To do this, we need to first create a session. This is an object which encapsulates the environment in which operations are exececuted:

In [10]:
sess = tf.Session()

We can then use this object's ```run``` method to execute our node.

In [11]:
sess.run(node_1)

3.0

We now need to close our session, to avoid using resources that may be required by other sessions.

In [13]:
sess.close()

Because of this, it is more common to construct sessions using the ```with``` statement:

In [14]:
with tf.Session() as sess:
    print(sess.run(node_1))

3.0


This automatically closes the session.

### 2.2 - Operations

Now we are going to build a very simple graph, which adds two numbers together. 

First create our constants:

In [15]:
node_1 = tf.constant(3.0, dtype=tf.float32, name='node_1')
node_2 = tf.constant(5.0, dtype=tf.float32, name='node_2')

Now, we are going to create a node to add these two nodes together:

In [16]:
node_3 = tf.add(node_1, node_2)

Remember, we have just built the graph at the moment. Now, let's run it:

In [18]:
with tf.Session() as sess:
    print(sess.run(node_3))

8.0


Our first (proper) graph works! At this stage it may not be clear exactly why we need all this code to add two numbers together, but hopefully it will become clearer shortly.

In [20]:
# OPTIONAL
# Multiply the two numbers together

### YOUR CODE HERE ###

### 2.3 - Placeholders